<a href="https://colab.research.google.com/github/Sciform/sciform-hwz-ai-in-controlling/blob/main/lecture_2/2_4_ts_04_forecasting_with_ml_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forecasting with machine learning

## Setup

In [ ]:
!pip install tensorflow==2.16.1

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import keras

# Print TensorFlow version
print(f"TensorFlow version: {tf.__version__}")

# Print Keras version
print(f"Keras version: {keras.__version__}")

In [ ]:
def plot_series(time, series, format="-", start=0, end=None, label=None):
    plt.plot(time[start:end], series[start:end], format, label=label)
    plt.xlabel("Time")
    plt.ylabel("Value")
    if label:
        plt.legend(fontsize=14)
    plt.grid(True)

def trend(time, slope=0):
    return slope * time


def seasonal_pattern(season_time):
    """Just an arbitrary pattern, you can change it if you wish"""
    return np.where(season_time < 0.4,
                    np.cos(season_time * 2 * np.pi),
                    1 / np.exp(3 * season_time))


def seasonality(time, period, amplitude=1, phase=0):
    """Repeats the same pattern at each period"""
    season_time = ((time + phase) % period) / period
    return amplitude * seasonal_pattern(season_time)


def white_noise(time, noise_level=1, seed=None):
    rnd = np.random.RandomState(seed)
    return rnd.randn(len(time)) * noise_level

In [ ]:
time = np.arange(4 * 365 + 1)

slope = 0.05
baseline = 10
amplitude = 40
series = baseline + trend(time, slope) + seasonality(time, period=365, amplitude=amplitude)

noise_level = 5
noise = white_noise(time, noise_level, seed=42)

series += noise

plt.figure(figsize=(10, 6))
plot_series(time, series)
plt.show()

## Forecasting with a "simple" Deep Learning Method

We choose the simplest Deep Learning Model possible, which consists
of 1 layer and 1 unit. This model is equivalent to linear regression.


First, we will train a model to forecast the next step given the previous 30 steps, therefore, we need to create a dataset of 30-step windows for training.

In [ ]:
def window_dataset(series, window_size, batch_size=32,
                   shuffle_buffer=1000):
    """
    Generate a data set in the format
    [X in window_size, y]
    [0,1,2,...., window_size-1, window_size]
    [1,2,3,...., window_size, window_size+1]
    ....
    """
    # convert ndarray into tensorflow data set format
    dataset = tf.data.Dataset.from_tensor_slices(series)
    # choose window size for X and 1 for y
    # shift by 1 to generate the next one and so on
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    # shuffle all dataset entries to break correlations
    dataset = dataset.shuffle(shuffle_buffer)
    # restore window : all X entries and then y
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

We split the data into a training and validation data set. (We omit the test set, which would be also necessary to run an independent test.)

In [ ]:
# do a sequential split here
# first - more historic - part of the time series is the training data
# second - more current - part of the time series is the validation data
# No random shuffling here, as we have to preserve the sequential order

split_time = 1000
time_train = time[:split_time]
x_train = series[:split_time]
time_valid = time[split_time:]
x_valid = series[split_time:]

### Linear Model

#### Linear model with an estimated learning rate

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

# choose windows size
window_size = 30

# create training data set X_i = {0,1,2,..., window_size-1} of length window_size, y = value of next time step
train_set = window_dataset(x_train, window_size)
# create validation data set
valid_set = window_dataset(x_valid, window_size)

# create "Deep Learning Model" with 1 layer and 1 unit (this is equivalent to a function with 1 parameter w = Linear Regression)
model = keras.models.Sequential([
  keras.layers.Input(shape=[window_size]),
  keras.layers.Dense(1), # , input_shape=[window_size])
])

# choose Stochastic Gradient Descent (SDG) to solve the resulting optimization problem
# lr : learning rate
optimizer = keras.optimizers.SGD(learning_rate=1e-4, momentum=0.9)

# collect the remaining information
# optimization or loss function = Huber loss
# metrics = mean absolute error (mae)
model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

# finally - train the model with 100 epochs and the validation set
model.fit(train_set, epochs=100, validation_data=valid_set)

#### Learning Rate Scheduler

The accuracy of the model is highly dependent on the learning rate. Now we use the Learning Rate Scheduler first to obtain an optimal learning rate and see whether we can improve the result.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30

train_set = window_dataset(x_train, window_size)

model = keras.models.Sequential([
  keras.layers.Input(shape=[window_size]),
  keras.layers.Dense(1) #, input_shape=[window_size])
])

# learning rate scheduler runs over a range of learning rates
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 30))

optimizer = keras.optimizers.SGD(learning_rate=1e-6, momentum=0.9)

model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])


# add callback with the learning rate scheduler
lr_opt_model = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

We can plot the loss now for every learning rate and pick the learning rate, where the loss appears to minimal but still stable.

In [ ]:
plt.semilogx(lr_opt_model.history["learning_rate"], lr_opt_model.history["loss"])
plt.axis([1e-6, 1e-3, 0, 20])

#### Linear model with optimized learning rate

Now we use the learning rate, which we determined above.

In [ ]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30
train_set = window_dataset(x_train, window_size)
valid_set = window_dataset(x_valid, window_size)

model = keras.models.Sequential([
   keras.layers.Input(shape=[window_size]),
  keras.layers.Dense(1), # input_shape=[window_size])
])

optimizer = keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9)

model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])

early_stopping = keras.callbacks.EarlyStopping(patience=10)
model.fit(train_set, epochs=500,
          validation_data=valid_set,
          callbacks=[early_stopping])

#### Forecast

In [ ]:
def model_forecast(model, series, window_size):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
# forecast based on linear deep learning model
lin_forecast = model_forecast(model, series[split_time - window_size:-1], window_size)[:, 0]

In [ ]:


plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, lin_forecast)

In [ ]:
keras.metrics.mean_absolute_error(x_valid, lin_forecast).numpy()

### Dense Model Forecasting

In [ ]:
# use the following model first - try to improve


# this model consists of 3 layer with 10 or 1 units and a nonlinear activation function called "relu"
model = keras.models.Sequential([
  keras.layers.Input(shape=[window_size]),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(1)
])



In [ ]:
# get inspired from the linear model

keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30

train_set = window_dataset(x_train, window_size)


# learning rate scheduler runs over a range of learning rates
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 30))

optimizer = keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9)

model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])


# add callback with the learning rate scheduler
lr_opt_model = model.fit(train_set, epochs=100, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(lr_opt_model.history["learning_rate"], lr_opt_model.history["loss"])
plt.axis([1e-6, 1e-3, 0, 20])

In [17]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

window_size = 30

train_set = window_dataset(x_train, window_size)

model = keras.models.Sequential([
  keras.layers.Input(shape=[window_size]),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(10, activation="relu"),
  keras.layers.Dense(1)
])

# learning rate scheduler runs over a range of learning rates
lr_schedule = keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-6 * 10**(epoch / 30))

optimizer = keras.optimizers.SGD(learning_rate=1e-5, momentum=0.9)

model.compile(loss=keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])


# add callback with the learning rate scheduler
lr_opt_model = model.fit(train_set, epochs=500, callbacks=[lr_schedule])

31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 18.2996 - mae: 18.7937 - learning_rate: 19.9526
Epoch 221/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 19.9770 - mae: 20.4725 - learning_rate: 21.5443
Epoch 222/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 17.4568 - mae: 17.9536 - learning_rate: 23.2631
Epoch 223/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 21.2951 - mae: 21.7893 - learning_rate: 25.1189
Epoch 224/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 17.8894 - mae: 18.3840 - learning_rate: 27.1227
Epoch 225/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 19.8015 - mae: 20.2931 - learning_rate: 29.2864
Epoch 226/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 18.1619 - mae: 18.6590 - learning_rate: 31.6228
Epoch 227/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 6506s 4ms/step - loss: 21.7594 - mae: 22.2514 - learning_rate: 34.1455
Epoch 228/500
31/31 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 22.9014 - mae: 23.3969 - learning_rate: 36.8694
Epoch 229/500
31/31 ━━━━━━━━━━

In [ ]:
# forecast based on linear deep learning model
dense_forecast = model_forecast(model, series[split_time - window_size:-1], window_size)[:, 0]

In [ ]:

plt.figure(figsize=(10, 6))
plot_series(time_valid, x_valid)
plot_series(time_valid, dense_forecast)

In [ ]:
keras.metrics.mean_absolute_error(x_valid, dense_forecast).numpy()

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# Source: https://github.com/tensorflow/examples